In [12]:
import torch
import torch.nn as nn
import numpy as np
import cv2
import pickle
from tqdm import tqdm
import random
import os
import glob
import sklearn
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt

In [13]:
dir_name = '/home/administrator/Sidhanth/topo_fe/pickled_data/'

def pickle_data(name,data,dir_name=dir_name):
    pickle_file = dir_name + name
    pickle_in = open(pickle_file,'wb')
    pickle.dump(data,pickle_in)
    pickle_in.close()
    
def pickle_load(pickle_file):
    pickle_out = open(pickle_file,'rb')
    data = pickle.load(pickle_out)
    pickle_out.close()
    return data
    
def load_pickle_file(name,img_size,dataset_path=None,dir_name=dir_name):
    path_string = '_' + str(img_size) + '.pickle'
    pickle_file = dir_name + name + path_string
    
    if os.path.exists(pickle_file):
        data = pickle_load(pickle_file)
        return data
    
    elif dataset_path is not None:
        def load_data(dataset_path=dataset_path):
            data = []
            index = 0
            for data_class in dataset_path:
                data_paths = sorted(glob.glob(data_class))
                class_num = dataset_path.index(data_class)
                print(class_num)
                count = 0
                for data_path in tqdm(data_paths):
                    try:
                        img = cv2.imread(data_path,1)
                        img = cv2.resize(img,(img_size,img_size),interpolation=cv2.INTER_CUBIC)
                        data.append([np.asarray(img),class_num])
                        count += 1
                    except Exception as e:
                        print(path+':Corrupt file')
                        pass
                print('Class '+str(class_num)+':'+str(count))
                index += 1
            return data
        
        dataset = load_data()
        random.shuffle(dataset)
        x = []
        y = []
        
        for images,labels in dataset:
            x.append(images)
            y.append(labels)
            
        x,y = np.array(x),np.array(y)
        print(x.shape,y.shape)
        pickle_data('x' + path_string,x)
        pickle_data('y' + path_string,y)
        data = pickle_load(pickle_file)
        return data

In [24]:
html_path = '/home/administrator/Sidhanth/topo_fe/mapper_outputs/'
path = '/home/administrator/Sidhanth/Datasets/cat_dog_dataset/training_set/'
cat_dataset_path = path + 'cats/cat.*.jpg'
dog_dataset_path = path + 'dogs/dog.*.jpg'
dataset_path = [cat_dataset_path,dog_dataset_path]

img_size = 32

x = load_pickle_file('x',img_size,dataset_path=dataset_path)
y = load_pickle_file('y',img_size)
x = np.moveaxis(x,-1,1)

x_train,x_test = x[:6400],x[6400:] 
y_train,y_test = y[:6400],y[6400:]

#x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)
#print(x.shape,y.shape,x_train.shape,y_train.shape)
x_,y_ = torch.from_numpy(x_train).float().cuda(),torch.from_numpy(y_train).float().cuda()


class cat_dog_CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 128, kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1)
        
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1)
        self.conv4 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1)
        
        self.conv5 = nn.Conv2d(256, 512, kernel_size=3, stride=2, padding=1)
        self.conv6 = nn.Conv2d(512, 1, kernel_size=3, stride=1, padding=1)

    def forward(self, xb):
        xb = xb.view(-1, 3, 32, 32)
        xb = F.leaky_relu(self.conv1(xb))
        xb = F.relu(self.conv2(xb))
        
        xb = F.leaky_relu(self.conv3(xb))
        xb = F.relu(self.conv4(xb))
        
        xb = F.leaky_relu(self.conv5(xb))
        xb = F.relu(self.conv6(xb))
        
        xb = F.avg_pool2d(xb, 4)
        xb = F.sigmoid(xb)
        return xb.view(-1)
    
model = cat_dog_CNN().cuda()
criterion_bce = nn.BCELoss()
opt = torch.optim.SGD(model.parameters(),lr=0.0001,momentum=0.9)

epochs = 500
batch_size = 128


for epoch in range(epochs):
    epoch_loss = 0
    start_index = 0
    end_index = 0
    
    #with tqdm(total=8000/batch_size) as pbar:
    while end_index < 6400:
        end_index = start_index + batch_size
        image = x_[start_index:end_index]
        pred = model(image)
        label = y_[start_index:end_index]
        model.zero_grad()
        loss = criterion_bce(pred,label)
        loss.backward()
        opt.step()
        epoch_loss += loss.item()
        #pbar.set_postfix_str(s='Loss:'+str(loss.item()))
        #pbar.update()
        start_index = end_index
    if epoch%5==0:
        print('Epoch:Loss',epoch+1,epoch_loss)

Epoch:Loss 1 34.76924389600754
Epoch:Loss 6 34.65751761198044
Epoch:Loss 11 34.65745663642883
Epoch:Loss 16 34.65742415189743
Epoch:Loss 21 34.65740633010864
Epoch:Loss 26 34.657394766807556
Epoch:Loss 31 34.65738445520401
Epoch:Loss 36 34.657374799251556
Epoch:Loss 41 34.65736651420593
Epoch:Loss 46 34.65736025571823
Epoch:Loss 51 34.65735387802124
Epoch:Loss 56 34.65734785795212
Epoch:Loss 61 34.65734225511551
Epoch:Loss 66 34.65733689069748
Epoch:Loss 71 34.65733140707016
Epoch:Loss 76 34.65732669830322
Epoch:Loss 81 34.65732169151306
Epoch:Loss 86 34.657316863536835
Epoch:Loss 91 34.65731346607208
Epoch:Loss 96 34.657310009002686
Epoch:Loss 101 34.65730655193329
Epoch:Loss 106 34.657303750514984
Epoch:Loss 111 34.657301008701324
Epoch:Loss 116 34.65729856491089
Epoch:Loss 121 34.65729594230652
Epoch:Loss 126 34.657293140888214
Epoch:Loss 131 34.65729033946991
Epoch:Loss 136 34.657287776470184
Epoch:Loss 141 34.657285153865814
Epoch:Loss 146 34.657282531261444
Epoch:Loss 151 34.6572

In [25]:
x_t,y_t = torch.from_numpy(x_test).float().cuda(),torch.from_numpy(y_test).float().cuda()

In [30]:
y_pred = model(x_t).cpu().detach().numpy()

In [33]:
y_pred[y_pred>0.5]

array([0.5002766 , 0.50007164, 0.5008787 , 0.50040317], dtype=float32)